In [1]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config
from random import seed, randint, random

# Initialize TensorFlow.
tflib.init_tf()

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

/opt/conda/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [9]:
# # Initialize TensorFlow.
# tflib.init_tf()

# Load pre-trained network.
url = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
    _G, _D, Gs = pickle.load(f)
        # _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
        # _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
        # Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.

In [10]:
# Pick latent vector.
def pick_latent_vec():
    value=randint(1,2**30)
    #alue = random()
    rnd = np.random.RandomState(value)
    latents = rnd.randn(1, Gs.input_shape[1])
    return latents

In [11]:
# Generate image.
def gen_image(latents):
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)
    return images

In [12]:
latent = pick_latent_vec()
image = gen_image(latent)
os.makedirs(config.result_dir, exist_ok=True)
png_filename = os.path.join(config.result_dir, 'exampleStylegan2.png')
PIL.Image.fromarray(image[0], 'RGB').save(png_filename)

In [ ]:
# Use this code to generate multiple images, set num_imgs to desired amount
# Additionally, change 
num_imgs = 180
i = 0
images = []
while i < num_imgs:
    latent = pick_latent_vec()
    image = gen_image(latent)
    os.makedirs(config.result_dir, exist_ok=True)
    png_filename = os.path.join(config.result_dir, 'longvidimg%s.png' %i)
    PIL.Image.fromarray(image[0], 'RGB').save(png_filename)
    i+=1

In [15]:
# This code loads the finetuned training performed on the deep dream and vgg16 visual reprsentations 
a = os.getcwd()
#model = a + '/results/00002-sgan-dream-4gpu/network-snapshot-007900.pkl'
model = a + '/results/00003-sgan-dream1024-4gpu/network-snapshot-007860.pkl'

with open(model, 'rb') as f:
    _G, _D, Gs = pickle.load(f)

In [17]:
# same code as cells above, but for producing finetuned images
latent = pick_latent_vec()
image = gen_image(latent)
os.makedirs(config.result_dir, exist_ok=True)
png_filename = os.path.join(config.result_dir, 'exampleFinetune2.png')
PIL.Image.fromarray(image[0], 'RGB').save(png_filename)

In [ ]:
# same code as cells above, but for producing finetuned images
num_imgs = 40
i = 0
images = []
while i < num_imgs:
    latent = pick_latent_vec()
    image = gen_image(latent)
    os.makedirs(config.result_dir, exist_ok=True)
    png_filename = os.path.join(config.result_dir, 'finetuneimg%s.png' %i)
    PIL.Image.fromarray(image[0], 'RGB').save(png_filename)
    i+=1